In [1]:
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import matplotlib.pyplot as plt
import os

os.environ['KMP_DUPLICATE_LIB_OK']="TRUE"

In [2]:
# 이미지넷(ImageNet) 데이터셋에서 추출한 사진들을 활용
# 120만 개가 넘는 이미지
from tensorflow.keras.applications import VGG16

In [3]:
BATCH_SIZE = 5

In [4]:
# 특징을 미묘하게 바꿔서
# 이미지 데이터의 수를 확장할 때 효과적으로 사용할 수 있다
train_datagen = ImageDataGenerator(rescale=1./255,          # 주어진 이미지의 크기를 설정합니다.
                                  horizontal_flip=True,     # 수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
                                  width_shift_range=0.1,    # 전체 크기의 15% 범위에서 좌우로 이동합니다.
                                  height_shift_range=0.1,   # 마찬가지로 위, 아래로 이동합니다.
                                  #rotation_range=5,        # 정해진 각도만큼 회전시킵니다.
                                  #shear_range=0.7,         # 좌표 하나를 고정시키고 나머지를 이동시킵니다.
                                  #zoom_range=1.2,          # 확대 또는 축소시킵니다.
                                  #vertical_flip=True,      # 수직 대칭 이미지를 만듭니다.
                                  #fill_mode='nearest'      # 빈 공간을 채우는 방법입니다. nearest 옵션은 가장 비슷한 색으로 채우게 됩니다.
                                  )

train_generator = train_datagen.flow_from_directory(
       './data/train',   # 학습셋이 있는 폴더의 위치
       target_size=(150, 150), # 이미지크기
       batch_size=BATCH_SIZE,
       class_mode='binary') # 치매/정상 이진 분류이므로 바이너리 모드로 실행

Found 160 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)  

test_generator = test_datagen.flow_from_directory(
       './data/test',   # 테스트셋이 있는 폴더의 위치
       target_size=(150, 150),
       batch_size=BATCH_SIZE,
       class_mode='binary')

Found 120 images belonging to 2 classes.


In [6]:
# include_top=False란
# VGG16의 마지막 층, 즉 분류를 담당하는 곳을 불러올지 말지를 정하는 옵션
# 우리가 만든 로컬 네트워크를 연결할 것이므로 False로 설정
transfer_model = VGG16(weights='imagenet', include_top=False, 
                      input_shape=(150, 150, 3))

transfer_model.trainable = False
transfer_model.summary()

58889256/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                              

In [7]:
# VGG16의 마지막 층을 우리가 만든 로컬 네트워크를 연결
finetune_model = models.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(layers.Flatten())
finetune_model.add(layers.Dense(64))
finetune_model.add(layers.Activation('relu'))
finetune_model.add(layers.Dropout(0.5))
finetune_model.add(layers.Dense(1))
finetune_model.add(layers.Activation('sigmoid'))

finetune_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 64)                524352    
                                                                 
 activation (Activation)     (None, 64)                0         
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
 activation_1 (Activation)   (None, 1)                 0

In [8]:
finetune_model.compile(loss='binary_crossentropy', 
              # 디폴트 learing_rate는 0.001 보폭을 더 줄인거임
              optimizer=optimizers.Adam(learning_rate=0.0002), 
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [9]:
history = finetune_model.fit(
    train_generator, epochs=20, 
    validation_data=test_generator, 
    callbacks=[early_stopping]
)

Epoch 1/20
32/32 [==============================] - 15s 451ms/step - loss: 0.6980 - accuracy: 0.5562 - val_loss: 0.5648 - val_accuracy: 0.8167
Epoch 2/20
32/32 [==============================] - 14s 438ms/step - loss: 0.5613 - accuracy: 0.6938 - val_loss: 0.4953 - val_accuracy: 0.8000
Epoch 3/20
32/32 [==============================] - 14s 444ms/step - loss: 0.4743 - accuracy: 0.8375 - val_loss: 0.4579 - val_accuracy: 0.7917
Epoch 4/20
32/32 [==============================] - 14s 449ms/step - loss: 0.4316 - accuracy: 0.7875 - val_loss: 0.4473 - val_accuracy: 0.8000
Epoch 5/20
32/32 [==============================] - 14s 438ms/step - loss: 0.3507 - accuracy: 0.9000 - val_loss: 0.3288 - val_accuracy: 0.9250
Epoch 6/20
32/32 [==============================] - 14s 435ms/step - loss: 0.3653 - accuracy: 0.8250 - val_loss: 0.3349 - val_accuracy: 0.8500
Epoch 7/20
32/32 [==============================] - 14s 447ms/step - loss: 0.3370 - accuracy: 0.8313 - val_loss: 0.2803 - val_accuracy: 0.9083